In [2]:
import os
import random

import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.nn.functional as F
import torchaudio.transforms as T
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset, DataLoader

from time import time
from tqdm import tqdm

from src.model.model import SpeakerCountCNN
import src.model.model as m


class SpectrogramDataset(Dataset):
    def __init__(self, csv_file, data_dir):

        self.data = pd.read_csv(csv_file)
        self.data_dir = data_dir
        self.labels = self.data['speaker_count'].astype(int).tolist()  # <- add this


    def __len__(self):
    
        return len(self.data)
    

    def __getitem__(self, idx):
    
        row = self.data.iloc[idx]
        tensor_path = os.path.join(self.data_dir, row['spectrogram'])
        spectrogram = torch.load(tensor_path).unsqueeze(0).float();  # shape: [1, H, W]
        label = int(row['speaker_count'])
        return spectrogram, label

dataset = SpectrogramDataset('data\spectrogram_labels_aug.csv','data\spectrograms_aug')

# Model with additional conv layer
class SpeakerCountCNN_test(nn.Module):
    def __init__(self, conv1_out, conv2_out, conv3_out, conv4_out, fc_hidden, dropout_prob, input_height=96, input_width=64, num_classes=6):
        super(SpeakerCountCNN_test, self).__init__()

        self.conv1 = nn.Conv2d(1, conv1_out, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(conv1_out)

        self.conv2 = nn.Conv2d(conv1_out, conv2_out, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(conv2_out)

        self.conv3 = nn.Conv2d(conv2_out, conv3_out, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(conv3_out)

        self.conv4 = nn.Conv2d(conv3_out, conv4_out, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(conv4_out)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(dropout_prob)

        # Dynamically compute flattened feature size
        with torch.no_grad():
            dummy = torch.zeros(1, 1, input_height, input_width)
            x = self.pool(F.relu(self.bn1(self.conv1(dummy))))
            x = self.pool(F.relu(self.bn2(self.conv2(x))))
            x = self.pool(F.relu(self.bn3(self.conv3(x))))
            x = self.pool(F.relu(self.bn4(self.conv4(x))))
            flattened_size = x.view(1, -1).shape[1]

        self.fc1 = nn.Linear(flattened_size, fc_hidden)
        self.fc2 = nn.Linear(fc_hidden, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = torch.flatten(x, 1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [5]:
# Data loaders for full data set
def get_loaders(dataset, train_frac=0.8, bs=32, seed=42):
    n = len(dataset)
    t = int(train_frac * n)
    train_ds, val_ds = random_split(dataset, [t, n-t], generator=torch.Generator().manual_seed(seed))
    return (DataLoader(train_ds, batch_size=bs, shuffle=True),
            DataLoader(val_ds, batch_size=bs, shuffle=False))



# Data loaders for subset, makes hp tuning quicker (stratified sampling prevents class imbalance)
def get_stratified_loaders(
    dataset,
    subset_frac: float = 0.3,
    train_frac: float = 0.8,
    batch_size: int = 32,
    random_state: int = 42,
    shuffle_train: bool = True
):

    # 1. Stratified sampling of the full dataset
    full_indices = list(range(len(dataset)))
    labels = dataset.labels
    subset_idx, _ = train_test_split(
        full_indices,
        train_size=subset_frac,
        stratify=labels,
        random_state=random_state
    )
    reduced_ds = Subset(dataset, subset_idx)

    # 2. Stratified train/val split of the reduced dataset
    reduced_labels = [labels[i] for i in subset_idx]
    train_idx, val_idx = train_test_split(
        list(range(len(reduced_ds))),
        train_size=train_frac,
        stratify=reduced_labels,
        random_state=random_state
    )

    # 3. Build DataLoaders
    train_loader = DataLoader(
        Subset(reduced_ds, train_idx),
        batch_size=batch_size,
        shuffle=shuffle_train
    )
    val_loader = DataLoader(
        Subset(reduced_ds, val_idx),
        batch_size=batch_size,
        shuffle=False
    )

    return train_loader, val_loader



# Model builder
def build_model(cfg, input_h, input_w):
    return SpeakerCountCNN(
        input_height=input_h, input_width=input_w,
        conv1_out=cfg['conv1_out'], conv2_out=cfg['conv2_out'],
        conv3_out=cfg['conv3_out'], fc_hidden=cfg['fc_hidden'],
        dropout_prob=cfg['dropout']
    )



# One‐epoch training with tqdm
from tqdm import tqdm
from time import time
def train_one_epoch(model, loader, opt, crit, device):
    model.train()
    total = 0
    for x, y in tqdm(loader, desc="Training", unit="batch"):
        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        loss = crit(model(x), y)
        loss.backward()
        opt.step()
        total += loss.item()
    return total / len(loader)



# Evaluate model
def validate(model, loader, device):
    model.eval()
    correct = 0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            preds = model(x).argmax(1)
            correct += (preds==y).sum().item()
    return correct / len(loader.dataset)

In [7]:
# 1. Dataset
dataset = SpectrogramDataset(csv_file="data/spectrogram_labels.csv", data_dir="data/spectrograms")
dataset_aug = SpectrogramDataset(csv_file="data/spectrogram_labels_aug.csv", data_dir="data/spectrograms_aug")

from torch.utils.data import ConcatDataset
dataset_comb = ConcatDataset([dataset,dataset_aug])


# 2. Model config and init
config = {
    'lr': 0.001,
    'dropout': 0.3,
    'fc_hidden': 128,
    'conv1_out': 32,
    'conv2_out': 16,
    'conv3_out': 64,
    'conv4_out': 64
}



model = SpeakerCountCNN_test(
    conv1_out=config['conv1_out'],
    conv2_out=config['conv2_out'],
    conv3_out=config['conv3_out'],
    conv4_out=config['conv4_out'],
    fc_hidden=config['fc_hidden'],
    dropout_prob=config['dropout'],
    input_height=96,
    input_width=64
)



# 3. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



# 4. DataLoaders

# train_loader, val_loader = get_stratified_loaders(
#     dataset, subset_frac=0.3, train_frac=0.8, batch_size=32, random_state=42
# )


train_loader, val_loader = get_loaders(dataset_comb)



# 5. Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['lr'])



# 6. Training loop with early stopping
# if loss doesn't improve for 3 epochs, stop training.
loss_list = []
strike_count = 0
threshold = 0.001

for epoch in range(50):
    start_time = time()
    avg_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    duration = time() - start_time
    loss_list.append(avg_loss)

    print(f"Epoch {epoch+1} — Loss: {avg_loss:.4f} — Time: {duration:.2f}s")

    if epoch > 0:
        if loss_list[epoch] < loss_list[epoch - 1] - threshold:
            strike_count = 0  # Reset on improvement
        else:
            strike_count += 1
            print(f"No significant improvement — strike {strike_count}/3")

        if strike_count == 3:
            print("Early stopping triggered after 3 consecutive low-improvement epochs.")
            break



# 7. Final Validation (refactored to use helper)
acc = validate(model, val_loader, device)
print(f"Validation Accuracy: {acc:.2%}")



# 8. Save model
torch.save(model.state_dict(), 'SpeakerCountCNN_comb_v1.pt')

Training: 100%|██████████| 1772/1772 [23:54<00:00,  1.24batch/s]


Epoch 1 — Loss: 0.5837 — Time: 1434.07s


Training: 100%|██████████| 1772/1772 [20:22<00:00,  1.45batch/s]


Epoch 2 — Loss: 0.4982 — Time: 1222.04s


Training: 100%|██████████| 1772/1772 [25:00<00:00,  1.18batch/s]


...

Epoch 48 — Loss: 0.1194 — Time: 1390.52s


Training: 100%|██████████| 1772/1772 [28:12<00:00,  1.05batch/s]


Epoch 49 — Loss: 0.1182 — Time: 1692.98s


Training: 100%|██████████| 1772/1772 [27:44<00:00,  1.06batch/s]


Epoch 50 — Loss: 0.1169 — Time: 1664.82s
Validation Accuracy: 83.13%


In [ ]:
import optuna

# SH trial over 5 epochs
def objective(trial):

    cfg = {
      'lr': trial.suggest_categorical('lr', [5e-5,1e-4,5e-4]),
      'dropout': trial.suggest_categorical('dropout', [0.25,0.3,0.35]),
      'fc_hidden': trial.suggest_categorical('fc_hidden',[64,128,256]),
      'conv1_out': trial.suggest_categorical('conv1_out',[16,32]),
      'conv2_out': trial.suggest_categorical('conv2_out',[16,32,64]),
      'conv3_out': trial.suggest_categorical('conv3_out',[64,128]),
      'conv4_out': trial.suggest_categorical('conv4_out',[32,64,128])
    }

    model = SpeakerCountCNN_test(
        conv1_out=cfg['conv1_out'],
        conv2_out=cfg['conv2_out'],
        conv3_out=cfg['conv3_out'],
        conv4_out=cfg['conv4_out'],
        fc_hidden=cfg['fc_hidden'],
        dropout_prob=cfg['dropout'],
        input_height=96,
        input_width=64
    )

    # Adam optim
    opt   = optim.Adam(model.parameters(), lr=cfg['lr'])

    # Cross entropy objective
    crit  = nn.CrossEntropyLoss()

    # Training and comparing to existing models
    for epoch in range(5):
        train_one_epoch(model, train_loader, opt, crit, device)
        val_acc = validate(model, val_loader, device)
        trial.report(val_acc, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_acc


# Creates a stratified subset, makes hp tuning quicker
train_loader, val_loader = get_stratified_loaders(
    dataset,
    subset_frac=0.3,   
    train_frac=0.8,    
    batch_size=32,
    random_state=42
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

study  = optuna.create_study(direction='maximize',
            pruner=optuna.pruners.SuccessiveHalvingPruner(), study_name='0-5_Speaker_4layer_HPTune')

study.optimize(objective, n_trials=100)

best_cfg = study.best_trial.params
print("Best HPs:", best_cfg)

[I 2025-07-26 23:39:56,823] A new study created in memory with name: 0-2_Speaker_4layer_HPTune
[I 2025-07-26 23:50:23,323] Trial 0 finished with value: 0.8416206261510129 and parameters: {'lr': 5e-05, 'dropout': 0.3, 'fc_hidden': 128, 'conv1_out': 16, 'conv2_out': 64, 'conv3_out': 128, 'conv4_out': 128}. Best is trial 0 with value: 0.8416206261510129.
[I 2025-07-27 00:03:14,606] Trial 1 finished with value: 0.8402394106813996 and parameters: {'lr': 0.0001, 'dropout': 0.35, 'fc_hidden': 256, 'conv1_out': 16, 'conv2_out': 16, 'conv3_out': 64, 'conv4_out': 128}. Best is trial 0 with value: 0.8416206261510129.
[I 2025-07-27 00:11:47,945] Trial 2 pruned. 
[I 2025-07-27 00:17:50,820] Trial 3 pruned. 
[I 2025-07-27 00:27:27,250] Trial 4 pruned. 
[I 2025-07-27 00:36:11,932] Trial 5 pruned. 
[I 2025-07-27 00:44:33,715] Trial 6 pruned. 
[I 2025-07-27 00:51:24,357] Trial 7 pruned. 
[I 2025-07-27 01:00:34,121] Trial 8 pruned. 
[I 2025-07-27 01:20:09,370] Trial 9 finished with value: 0.848066298342

Best HPs: {'lr': 0.0005, 'dropout': 0.3, 'fc_hidden': 128, 'conv1_out': 16, 'conv2_out': 64, 'conv3_out': 128, 'conv4_out': 128}


In [53]:
study.best_trial.value

0.85451197053407

In [55]:
# Model with config from Optuna study
cfg = {'lr': 0.0005, 'dropout': 0.3, 'fc_hidden': 128, 'conv1_out': 16, 'conv2_out': 64, 'conv3_out': 128, 'conv4_out': 128}


final_model = build_model(cfg, 96, 64).to(device)



# Adam optimizer
opt = optim.Adam(final_model.parameters(), lr=best_cfg['lr'])



# Cross Entropy loss
crit = nn.CrossEntropyLoss()



# Loaders
train_loader, val_loader = get_loaders(dataset)



# Train
num_epochs = 50
for epoch in range(num_epochs):
    t0 = time()
    loss = train_one_epoch(final_model, train_loader, opt, crit, device)  # tqdm is inside this
    dur = time() - t0
    print(f"Epoch {epoch+1} — Loss {loss:.4f} — {dur:.1f}s")



# Validate
val_acc = validate(final_model, val_loader, device)
print(f"Final val accuracy: {val_acc:.2%}")
torch.save(final_model.state_dict(), r"src/model/best_0-2_3layer.pt")

Training: 100%|██████████| 905/905 [11:16<00:00,  1.34batch/s]


Epoch 1 — Loss 0.5115 — 676.3s


Training: 100%|██████████| 905/905 [15:15<00:00,  1.01s/batch]


Epoch 2 — Loss 0.4429 — 915.4s


Training: 100%|██████████| 905/905 [15:19<00:00,  1.02s/batch]


...

Epoch 48 — Loss 0.0406 — 883.0s


Training: 100%|██████████| 905/905 [14:58<00:00,  1.01batch/s]


Epoch 49 — Loss 0.0345 — 898.4s


Training: 100%|██████████| 905/905 [15:13<00:00,  1.01s/batch]


Epoch 50 — Loss 0.0416 — 913.7s
Final val accuracy: 86.05%


## repeating with large epochs (early stopping on val accuracy)

WTF